In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

# Model with Keras Functional API

Functional API gives a lot of flexibility like: Multi-Input and Multi-Output model, Non-Sequential model, etc.

- `tf.keras.layers`: Devine layers to create model. Each layer is Callable object. It takes input tensor and return a tensor.
- `tf.keras.Input`: Defind input placeholder. Need to provide dimension of input data.
- `tf.keras.Model`: Take Input placeholder and Output Layer then return a Model.

Lets build a simple FCN with 2 layers and 10 output units.

In [2]:
inputs = tf.keras.Input(shape=(32,))  # Input shape: n_sample x 32 features
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
y = tf.keras.layers.Dense(10, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=y)

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 6,922
Trainable params: 6,922
Non-trainable params: 0
_________________________________________________________________


In [4]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
metrics = ['accuracy']
model.compile(optimizer=opt, loss=loss, metrics=metrics)

Lets cook some data to test if this works. Then train model on dummy data.

In [5]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

In [6]:
model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 11.5768 - accuracy: 0.0960 - val_loss: 11.5768 - val_accuracy: 0.0600
Epoch 2/10
1000/1000 [==============================] - 0s 237us/sample - loss: 11.5643 - accuracy: 0.0910 - val_loss: 11.5755 - val_accuracy: 0.0800
Epoch 3/10
1000/1000 [==============================] - 0s 286us/sample - loss: 11.5626 - accuracy: 0.1170 - val_loss: 11.5772 - val_accuracy: 0.0800
Epoch 4/10
1000/1000 [==============================] - 0s 229us/sample - loss: 11.5623 - accuracy: 0.1080 - val_loss: 11.5778 - val_accuracy: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 229us/sample - loss: 11.5616 - accuracy: 0.1210 - val_loss: 11.5766 - val_accuracy: 0.0800
Epoch 6/10
1000/1000 [==============================] - 0s 291us/sample - loss: 11.5582 - accuracy: 0.1470 - val_loss: 11.5812 - val_accuracy: 0.0700
Epoch 7/10
1000/1000 [==============================] -

To make prediction, just simply call `predict` function.

In [7]:
val_pred = model.predict(val_data)

In [8]:
val_pred.shape

(100, 10)

In [9]:
val_pred[:3, :]

array([[0.5433044 , 0.53207   , 0.52619827, 0.49454835, 0.5145661 ,
        0.4734469 , 0.4389014 , 0.5217423 , 0.53398573, 0.54983544],
       [0.5032129 , 0.46517012, 0.56516147, 0.56238633, 0.39433834,
        0.5217385 , 0.47368962, 0.55531204, 0.4832793 , 0.59997207],
       [0.54781497, 0.5588869 , 0.5316051 , 0.4838442 , 0.5425331 ,
        0.5947191 , 0.48622772, 0.5588054 , 0.44471452, 0.54464275]],
      dtype=float32)

# How to export a trained model

Tensorflow supports severl export methods like: Export as HDF5, Export as Tensorflow format which later can be used by Tensorflow Serving, export model's weights only as Numpy array, export model's architecture only.

Lets export model as Tensorflow format first. Then recreate model from saved file. Then check if 2 models version give same prediction on same input.

In [10]:
# From Tensorflow:
# Reset metrics before saving so that loaded model has same state,
# since metric states are not preserved by Model.save_weights
model.reset_metrics()

In [11]:
!mkdir -p ./temp

In [12]:
model.save('./temp/model', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./temp/model/assets


In [13]:
!ls ./temp

model


In [14]:
deployed_model = tf.keras.models.load_model('./temp/model')

In [15]:
new_val_pred = deployed_model.predict(val_data)
new_val_pred.shape

(100, 10)

In [16]:
np.testing.assert_allclose(val_pred, new_val_pred, rtol=1e-6, atol=1e-6)

Export model as Tensorflow format also keeps optimizer state. We can use this model and continue to train it on new data.

# Feed data into model

## Dataset

If input data fit well on Memory, the easy way to feed data into Model is just call `fit` function on data.

However, sometime we want to do some transformation on input data like: batching, shuffling, etc., we should use `tf.data` API. As this API provide a lot of functions to deal with various data formats and transformation.

Key Data Structure to know is `tf.data.Dataset`. It supports reading input data from source, apply transformation and a way to iterate over elements.

Lets start by making a Dataset from numpy array.

In [17]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.random((5, 2)))
dataset

<TensorSliceDataset shapes: (2,), types: tf.float64>

In [18]:
for element in dataset:
    print(element)

tf.Tensor([0.30119929 0.9186914 ], shape=(2,), dtype=float64)
tf.Tensor([0.33149118 0.34305358], shape=(2,), dtype=float64)
tf.Tensor([0.02524145 0.60788564], shape=(2,), dtype=float64)
tf.Tensor([0.24681797 0.96657151], shape=(2,), dtype=float64)
tf.Tensor([0.57920191 0.29299168], shape=(2,), dtype=float64)


Lets make a bigger dataset to test transformation. We will create 500 sample datasets. Instead of reading each element, we read 32 samples in a batch, and repeat for 3 epochs. Beside that, we also make some shuffling input data.

In [19]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.random((500, 2)))
dataset

<TensorSliceDataset shapes: (2,), types: tf.float64>

In [20]:
shuffle_sz = 100
batch_sz = 32
n_epochs = 3
dataset = dataset.shuffle(shuffle_sz).batch(batch_sz).repeat(n_epochs)
dataset

<RepeatDataset shapes: (None, 2), types: tf.float64>

Inspect the output of dataset. Just take 1 batch and confirm the number of sample is 32 and number of features is 2.

In [21]:
for data in dataset.take(1):
    break

In [22]:
data.shape

TensorShape([32, 2])

In [23]:
data

<tf.Tensor: shape=(32, 2), dtype=float64, numpy=
array([[0.78476127, 0.9730055 ],
       [0.05487506, 0.48777445],
       [0.04188906, 0.14167223],
       [0.97178323, 0.32838761],
       [0.30653912, 0.98024528],
       [0.45081555, 0.12278334],
       [0.01059869, 0.15662154],
       [0.74778029, 0.91978481],
       [0.17837831, 0.42350351],
       [0.39130799, 0.11854122],
       [0.67327952, 0.83489706],
       [0.08303975, 0.52951504],
       [0.17876153, 0.37055573],
       [0.1312979 , 0.97822332],
       [0.34868595, 0.1095313 ],
       [0.8120671 , 0.52496514],
       [0.07417451, 0.50987169],
       [0.53942034, 0.06101324],
       [0.41219032, 0.11949338],
       [0.06813085, 0.43337402],
       [0.1196409 , 0.73272511],
       [0.49920861, 0.04781624],
       [0.11695124, 0.52681623],
       [0.68876446, 0.34329967],
       [0.42922652, 0.90481399],
       [0.19912923, 0.56439666],
       [0.97466749, 0.34463219],
       [0.85192938, 0.90415445],
       [0.58065286, 0.56578

## Sparsed Dataset

When data is sparsed, turning data into Densed numpy array before making a Dataset is not a good way. Most of the time, we will run out of memory. A better way is to create a `SparsedTensor` then feed this tensor to Dataset. 

`SparsedTensor` take 3 inputs:

1. Cordinate where value exists.
2. Value to fill into above cordinates.
3. Dense shape.

For example, in below densed matrix,

```
col = 0  1  2  3
r0  [[1, 0, 0, 0]
r    [0, 0, 2, 0]
r    [0, 3, 0, 4]]
```

the input will be:

1. `row 0, col 0`, `row 1, col 2`, `row 2, col 1`, `row 2, col3` have values.
2. Values to fill are: `1, 2, 3, 4`
3. Densed shape is: `3 rows, 4 cols`

Lets make a `SparsedTensor` from above matrix. Then create a Dataset from it.

In [24]:
tensor = tf.sparse.SparseTensor(
    indices=[[0, 0], [1, 2], [2, 1], [2, 3]],
    values=[1, 2, 3, 4],
    dense_shape=[3, 4])
tensor

In [25]:
sparse_ds = tf.data.Dataset.from_tensor_slices(tensor)
sparse_ds

<TensorSliceDataset shapes: (4,), types: tf.int32>

In [26]:
for element in sparse_ds:
    print(element, end='\n\n')

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([1], shape=(1,), dtype=int32), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64))

SparseTensor(indices=tf.Tensor([[2]], shape=(1, 1), dtype=int64), values=tf.Tensor([2], shape=(1,), dtype=int32), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64))

SparseTensor(indices=tf.Tensor(
[[1]
 [3]], shape=(2, 1), dtype=int64), values=tf.Tensor([3 4], shape=(2,), dtype=int32), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64))



Another way is to create `SparsedTensor` from `scipy.sparse` data. Lets create a sparse data as below:

- `4 at (0, 0)`
- `5 at (3, 3)`
- `7 at (3, 1)`
- `9 at (2, 2)`

In [29]:
from scipy.sparse import coo_matrix


data = np.array([4, 5, 7, 9])
row  = np.array([0, 3, 3, 2])
col  = np.array([0, 3, 1, 2])
sparse_data = coo_matrix((data, (row, col)), shape=(4, 4))
sparse_data

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in COOrdinate format>

In [30]:
sparse_data.toarray()

array([[4, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 9, 0],
       [0, 7, 0, 5]])

In [31]:
sparse_data.row

array([0, 3, 3, 2], dtype=int32)

In [32]:
sparse_data.col

array([0, 3, 1, 2], dtype=int32)

In [33]:
sparse_data.data

array([4, 5, 7, 9])

In [34]:
sparse_data.shape

(4, 4)

In [35]:
tensor = tf.sparse.reorder(tf.sparse.SparseTensor(
    indices=[(r, c) for r, c in zip(sparse_data.row, sparse_data.col)],
    values=sparse_data.data,
    dense_shape=sparse_data.shape
))
tensor

In [36]:
sparse_ds = tf.data.Dataset.from_tensor_slices(tensor)
for element in sparse_ds:
    print(element, '\n')

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([4], shape=(1,), dtype=int64), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64)) 

SparseTensor(indices=tf.Tensor([], shape=(0, 1), dtype=int64), values=tf.Tensor([], shape=(0,), dtype=int64), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64)) 

SparseTensor(indices=tf.Tensor([[2]], shape=(1, 1), dtype=int64), values=tf.Tensor([9], shape=(1,), dtype=int64), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64)) 

SparseTensor(indices=tf.Tensor(
[[1]
 [3]], shape=(2, 1), dtype=int64), values=tf.Tensor([7 5], shape=(2,), dtype=int64), dense_shape=tf.Tensor([4], shape=(1,), dtype=int64)) 



*Remark*: Indices for `SparseTensor` must be in row order. Otherwise, it will raise an exception. To fix this, we wrap `tf.sparse.reorder()` around Tensor initialization. Our input to sparsed matrix is not row order.

## Dataset with Feature and Label

We will load a structured data to pandas, then create dataset from this data. Next, we will check how to train the model on dataset.

In [38]:
import pandas as pd

train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
val = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = train.pop('survived')
y_eval = val.pop('survived')

In [39]:
train.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [40]:
train.shape

(627, 9)

In [54]:
ds = tf.data.Dataset.from_tensor_slices((
    train[['fare', 'age', 'n_siblings_spouses', 'parch']].values,
    y_train.values))

In [55]:
ds

<TensorSliceDataset shapes: ((4,), ()), types: (tf.float64, tf.int64)>

In [56]:
shuffle_sz = 300
batch_sz = 8 
ds = ds.shuffle(shuffle_sz).batch(batch_sz)
ds

<BatchDataset shapes: ((None, 4), (None,)), types: (tf.float64, tf.int64)>

In [57]:
for x, y in ds.take(3):
    print(x.shape, y.shape)

(8, 4) (8,)
(8, 4) (8,)
(8, 4) (8,)


In [58]:
x

<tf.Tensor: shape=(8, 4), dtype=float64, numpy=
array([[29.125 ,  2.    ,  4.    ,  1.    ],
       [90.    , 44.    ,  2.    ,  0.    ],
       [25.4667, 28.    ,  3.    ,  1.    ],
       [ 7.8542, 28.    ,  0.    ,  0.    ],
       [ 7.925 , 39.    ,  0.    ,  0.    ],
       [ 7.225 , 28.    ,  0.    ,  0.    ],
       [ 7.05  , 35.    ,  0.    ,  0.    ],
       [14.4542, 14.5   ,  1.    ,  0.    ]])>

In [59]:
y

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 0, 0, 0, 1, 1, 0, 0])>

In [60]:
inputs = tf.keras.Input(shape=(4,))
x = tf.keras.layers.Dense(4, activation='relu')(inputs)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=y)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [61]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
model.compile(optimizer=opt, loss=loss, metrics=metrics)

model.fit(ds, epochs=3)

Train for 79 steps
Epoch 1/3
79/79 [==============================] - 1s 14ms/step - loss: 4.6461e-08 - accuracy: 0.3876
Epoch 2/3
79/79 [==============================] - 0s 6ms/step - loss: 4.5840e-08 - accuracy: 0.3876
Epoch 3/3
79/79 [==============================] - 1s 8ms/step - loss: 4.6461e-08 - accuracy: 0.3876


For structured data, `scikit-learn` framework or Gradient Boosting frame work work really well. However, it is not so hard to apply Deep Leaning on these problem with Keras API.

# Clean up

In [62]:
!rm -rf ./temp